In [152]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error as MSE, mean_absolute_error as MAE
from joblib import dump, load
import joblib
import random


In [153]:

# Фиксируем датчики случайных чисел
random.seed(42)
np.random.seed(42)

# Загружаем данные
df = pd.read_csv('data.csv')

# Разделяем ИСХОДНЫЕ ДАННЫЕ на тренировочную и тестовую выборки
df_train, df_test = train_test_split(df, train_size=0.75, random_state=42)

/var/folders/34/9lmn_1px5cs289tjl5rn7cs80000gn/T/ipykernel_3739/2594801504.py:6: DtypeWarning: Columns (33,37,38,46) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [154]:
df_kladr = pd.read_csv('kladr-2.csv')
df_kladr = df_kladr.sort_values("TYPE_NP")
df_kladr = df_kladr.drop_duplicates(keep='first', subset='NAME').sort_values('NAME')

In [155]:
alphabet = {}
for i, j in enumerate(["A", "B", "C", "D", "E", "F", "G", "H", "I", "K"]):
    alphabet[i] = j

In [156]:
df_kladr["OKRUG_CODE"] = df_kladr["OKRUG_CODE"].replace(alphabet)

In [157]:
df_kladr.loc[39998, "SOCR"] = 'г'

In [158]:
df_train = df_train.merge(df_kladr[['NAME', 'REGION', 'TYPE_NP', 'OKRUG_OCATD', 'OKRUG_CODE']], left_on='ann_city', right_on='NAME', how='left').drop(columns=['NAME'])
df_train.rename(columns={'TYPE_NP': 'type_city',
                   'REGION': 'region',
                   'OKRUG_OCATD': 'okr_ocatd',
                   'OKRUG_CODE': 'okr_code'
                   }, inplace=True)

In [159]:
df_test = df_test.merge(df_kladr[['NAME', 'REGION', 'TYPE_NP', 'OKRUG_OCATD', 'OKRUG_CODE']], left_on='ann_city', right_on='NAME', how='left').drop(columns=['NAME'])
df_test.rename(columns={'TYPE_NP': 'type_city',
                   'REGION': 'region',
                   'OKRUG_OCATD': 'okr_ocatd',
                   'OKRUG_CODE': 'okr_code'
                   }, inplace=True)

In [160]:
df_train["type_city"] = df_train["type_city"].fillna("Nan")
df_test["type_city"] = df_test["type_city"].fillna("Nan")

In [161]:
df_town = pd.read_excel('town.xlsx')

In [162]:
import pandas as pd

# Добавляем данные из df_3 по численности
df_train = df_train.merge(df_town[['Town', 'Count']], left_on='ann_city', right_on='Town', how='left').drop(columns=['Town'])

# Заменяем значения в type_city по условиям Count
df_train.loc[(df_train['Count'] >= 50) & (df_train['Count'] < 100), 'type_city'] = 'gor_100'
df_train.loc[(df_train['Count'] >= 100) & (df_train['Count'] < 250), 'type_city'] = 'gor_250'
df_train.loc[(df_train['Count'] >= 250) & (df_train['Count'] < 500), 'type_city'] = 'gor_500'
df_train.loc[(df_train['Count'] >= 500) & (df_train['Count'] < 1000), 'type_city'] = 'gor_1000'
df_train.loc[(df_train['Count'] >= 1000) & (df_train['Count'] < 2500), 'type_city'] = 'gor_2500'
df_train.loc[(df_train['Count'] >= 2500) & (df_train['Count'] < 5000), 'type_city'] = 'gor_5000'
df_train.loc[df_train['Count'] > 5000, 'type_city'] = 'gor_10000'
# df_train = df_train.drop(columns=['Count'])

In [163]:
import pandas as pd

# Добавляем данные из df_3 по численности
df_test = df_test.merge(df_town[['Town', 'Count']], left_on='ann_city', right_on='Town', how='left').drop(columns=['Town'])

# Заменяем значения в type_city по условиям Count
df_test.loc[(df_test['Count'] >= 50) & (df_test['Count'] < 100), 'type_city'] = 'gor_100'
df_test.loc[(df_test['Count'] >= 100) & (df_test['Count'] < 250), 'type_city'] = 'gor_250'
df_test.loc[(df_test['Count'] >= 250) & (df_test['Count'] < 500), 'type_city'] = 'gor_500'
df_test.loc[(df_test['Count'] >= 500) & (df_test['Count'] < 1000), 'type_city'] = 'gor_1000'
df_test.loc[(df_test['Count'] >= 1000) & (df_test['Count'] < 2500), 'type_city'] = 'gor_2500'
df_test.loc[(df_test['Count'] >= 2500) & (df_test['Count'] < 5000), 'type_city'] = 'gor_5000'
df_test.loc[df_test['Count'] > 5000, 'type_city'] = 'gor_10000'
# df_test = df_test.drop(columns=['Count'])

In [164]:
# Загрузка (если нужно)
rf_pipeline = load("rf_pipeline.joblib")

# Обучение Pipeline
transformed_train = rf_pipeline.fit_transform(df_train)

# Преобразование тестовых данных
transformed_test = rf_pipeline.transform(df_test)

In [165]:
# Преобразуем и разделяем признаки и таргет
X_train = transformed_train.copy()
y_train = transformed_train['car_price']
X_test = transformed_test.copy()
y_test = transformed_test['car_price']
X_train = X_train.drop(['car_price'], axis=1)
X_test = X_test.drop(['car_price'], axis=1)

In [166]:
rf_model = joblib.load('random_forest_pipeline_without_log.pkl')

In [167]:
y_pred_rf = rf_model.predict(X_test)
y_pred_train_rf = rf_model.predict(X_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 12, 13, 14, 15, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 10, 12, 13, 14, 15, 17] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [168]:
# Функция вывода основных характеристик модели (ориентируемся прежде всего на R^2)
def print_r2_mse(y_train, y_pred_train, y_test, y_pred):
    '''
    Функция принимает на вход признаки и таргет для тренировочных
    и тестовых данных, модель и выводит на экран 
    значения MSE, R^2 для трейна и теста для указанных параметров
    '''
    print('*'*20)
    print(f'Значение MAE для трейна:  {MAE(y_train, y_pred_train)}')
    print(f'Значение MAE для теста:   {MAE(y_test, y_pred)}')
    print('*'*20)
    print(f'Значение MSE для трейна:  {MSE(y_train, y_pred_train)}')
    print(f'Значение MSE для теста:   {MSE(y_test, y_pred)}')
    print('*'*20)
    print(f'Значение R^2 для трейна:  {r2_score(y_train, y_pred_train)}')
    print(f'Значение R^2 для теста:   {r2_score(y_test, y_pred)}')

In [169]:
print_r2_mse(y_train, y_pred_train_rf, y_test, y_pred_rf)

********************
Значение MAE для трейна:  318444.4522253497
Значение MAE для теста:   342453.1960682919
********************
Значение MSE для трейна:  3754516993926.684
Значение MSE для теста:   1590771116211.9438
********************
Значение R^2 для трейна:  0.9069304227277237
Значение R^2 для теста:   0.957044645738207


In [170]:
# Загрузка (если нужно)
loaded_pipeline = load("pipeline.joblib")

# Обучение Pipeline
transformed_train = loaded_pipeline.fit_transform(df_train)

# Преобразование тестовых данных
transformed_test = loaded_pipeline.transform(df_test)

In [171]:
# Преобразуем и разделяем признаки и таргет
X_train = transformed_train.copy()
y_train = transformed_train['car_price']
X_test = transformed_test.copy()
y_test = transformed_test['car_price']
X_train = X_train.drop(['car_price'], axis=1)
X_test = X_test.drop(['car_price'], axis=1)

In [172]:
# Преобразование категориальных признаков в Pandas category
for col in X_train.select_dtypes(include=['object']).columns:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

In [173]:


# Загрузка модели
lgbm_model = joblib.load('lgbm_pipeline_.pkl')
catboost_model = joblib.load('catboost_pipeline_.pkl')
ridge_model = joblib.load('ridge_pipeline_.pkl')


In [174]:
y_pred_lgbm = lgbm_model.predict(X_test)
y_pred_train_lgbm = lgbm_model.predict(X_train)

y_pred_cb = catboost_model.predict(X_test)
y_pred_train_cb = catboost_model.predict(X_train)

y_pred_ridge = ridge_model.predict(X_test)
y_pred_train_ridge = ridge_model.predict(X_train)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [175]:
y_pred = (y_pred_lgbm + y_pred_cb + np.log(y_pred_rf + 2900000) + y_pred_ridge) / 4
y_pred_train = (y_pred_train_lgbm + y_pred_train_cb + np.log(y_pred_train_rf + 2900000) + y_pred_train_ridge) / 4

In [176]:
# Функция вывода основных характеристик модели (ориентируемся прежде всего на R^2)
def print_r2_mse_exp(y_train, y_pred_train, y_test, y_pred):
    '''
    Функция принимает на вход признаки и таргет для тренировочных
    и тестовых данных, модель и выводит на экран 
    значения MSE, R^2 для трейна и теста для указанных параметров
    '''

    delta_y = 2900000
    
    print('*'*20)
    print(f'Значение MAE для трейна:  {MAE(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение MAE для теста:   {MAE(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')
    print('*'*20)
    print(f'Значение MSE для трейна:  {MSE(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение MSE для теста:   {MSE(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')
    print('*'*20)
    print(f'Значение R^2 для трейна:  {r2_score(np.exp(y_train) - delta_y, np.exp(y_pred_train) - delta_y)}')
    print(f'Значение R^2 для теста:   {r2_score(np.exp(y_test) - delta_y, np.exp(y_pred) - delta_y)}')

In [177]:
print_r2_mse_exp(y_train, y_pred_train_lgbm, y_test, y_pred_lgbm) # Выводим результаты, довольны собой

********************
Значение MAE для трейна:  217884.09450490336
Значение MAE для теста:   315298.71649147646
********************
Значение MSE для трейна:  522908576983.93
Значение MSE для теста:   1875556231847.2312
********************
Значение R^2 для трейна:  0.9870377786834723
Значение R^2 для теста:   0.9493546359021411


In [178]:
print_r2_mse_exp(y_train, y_pred_train_cb, y_test, y_pred_cb)

********************
Значение MAE для трейна:  284415.01216352795
Значение MAE для теста:   329634.497481051
********************
Значение MSE для трейна:  986582424747.463
Значение MSE для теста:   1488185306143.2114
********************
Значение R^2 для трейна:  0.9755439090130547
Значение R^2 для теста:   0.9598147550071185


In [179]:
print_r2_mse_exp(y_train, y_pred_train_ridge, y_test, y_pred_ridge)

********************
Значение MAE для трейна:  320394.2988547338
Значение MAE для теста:   349860.04969869717
********************
Значение MSE для трейна:  1522473707741.2415
Значение MSE для теста:   1606172021165.1626
********************
Значение R^2 для трейна:  0.9622598633547697
Значение R^2 для теста:   0.956628777407763


In [180]:
print_r2_mse_exp(y_train, y_pred_train, y_test, y_pred) # Выводим результаты, довольны собой

********************
Значение MAE для трейна:  260350.32601655737
Значение MAE для теста:   304979.6955741105
********************
Значение MSE для трейна:  1289575400244.5518
Значение MSE для теста:   1384662548767.2693
********************
Значение R^2 для трейна:  0.9680331085048671
Значение R^2 для теста:   0.9626101645238757
